<a href="https://colab.research.google.com/github/Dokuqui/FakeOutDetector/blob/main/Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

base_dir = '/content/drive/MyDrive/FakeOutDetector/data/processed'
real_dir = os.path.join(base_dir, 'real')
fake_dir = os.path.join(base_dir, 'fake')

# Load images  and labels
images = []
labels = [] # 0 = real, 1 = fake

for i in range(50):
  real_img = cv2.imread(os.path.join(real_dir, f'real_{i}.jpg'))
  fake_img = cv2.imread(os.path.join(fake_dir, f'fake_{i}.jpg'))
  if real_img is not None:
    images.append(real_img)
    labels.append(0)
  if fake_img is not None:
    images.append(fake_img)
    labels.append(1)

images = np.array(images)
labels = np.array(labels)

# Split into train (80%) and validation (20%)
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)
print(f"Training samples: {len(X_train)}, Validation samples: {len(X_val)}")

Training samples: 80, Validation samples: 20


In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary output: real (0) or fake (1)
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 52, 52, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 173056)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │      11,075,648 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,132,033 (42.47 MB)

 Trainable params: 11,132,033 (42.47 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
history = model.fit(X_train, y_train,
                    epochs=10,
                    batch_size=16,
                    validation_data=(X_val, y_val))

Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.5344 - loss: 375.8296 - val_accuracy: 0.6000 - val_loss: 6.7550
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.4602 - loss: 10.0570 - val_accuracy: 0.4000 - val_loss: 12.0827
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.5259 - loss: 4.1537 - val_accuracy: 0.4000 - val_loss: 1.4062
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 980ms/step - accuracy: 0.6219 - loss: 0.7374 - val_accuracy: 0.7000 - val_loss: 0.5927
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.8809 - loss: 0.3382 - val_accuracy: 0.9000 - val_loss: 0.3363
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 987ms/step - accuracy: 0.9762 - loss: 0.1698 - val_accuracy: 0.9500 - val_loss: 0.1852
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.9632 - loss: 0.1367 - val_accuracy: 0.9500 - val_loss: 0.2166
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.9710 - loss: 0.0749 - val_accuracy: 0.7500 - val_loss: 0.4941
Epo

In [5]:
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 355ms/step - accuracy: 0.6000 - loss: 0.9892
Validation Loss: 0.9891664385795593, Validation Accuracy: 0.6000000238418579


In [6]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=2, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_val, y_val), callbacks=[early_stopping])

Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.9113 - loss: 0.2949 - val_accuracy: 0.8500 - val_loss: 0.3786
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.9845 - loss: 0.1023 - val_accuracy: 0.8000 - val_loss: 0.3692
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 1.0000 - loss: 0.0759 - val_accuracy: 0.9500 - val_loss: 0.1295
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.9639 - loss: 0.0976 - val_accuracy: 0.9000 - val_loss: 0.2121
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.9691 - loss: 0.0899 - val_accuracy: 0.6000 - val_loss: 0.8070


In [7]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),  # Add dropout
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 52, 52, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 173056)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │      11,075,648 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,132,033 (42.47 MB)

 Trainable params: 11,132,033 (42.47 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
import tensorflow as tf
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False
model = tf.keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_val, y_val))

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 798ms/step - accuracy: 0.4184 - loss: 0.8467 - val_accuracy: 0.5000 - val_loss: 0.7093
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 709ms/step - accuracy: 0.5417 - loss: 0.7263 - val_accuracy: 0.6000 - val_loss: 0.6577
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 488ms/step - accuracy: 0.5059 - loss: 0.6896 - val_accuracy: 0.7500 - val_loss: 0.6194
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 511ms/step - accuracy: 0.6637 - loss: 0.6336 - val_accuracy: 0.7500 - val_loss: 0.5908
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 511ms/step - accuracy: 0.7247 - loss: 0.6027 - val_accuracy: 0.8500 - val_loss: 0.5924
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 667ms/step - accuracy: 0.7528 - loss: 0.5735 - val_accuracy: 0.9000 - val_loss: 0.5688
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 465ms/step - accuracy: 0.8108 - loss: 0.5678 - val_accuracy: 0.8500 - val_loss: 0.5523
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 518ms/step - accuracy: 0.8097 

In [9]:
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 528ms/step - accuracy: 0.9000 - loss: 0.5241
Validation Loss: 0.5240510702133179, Validation Accuracy: 0.8999999761581421


In [11]:
model_dir = '/content/drive/MyDrive/FakeOutDetector/models'
os.makedirs(model_dir, exist_ok=True)
model.save(os.path.join(model_dir, 'deepfake_model.h5'))  # Legacy HDF5
model.save(os.path.join(model_dir, 'deepfake_model.keras'))  # New Keras format

print(f"HDF5 saved: {os.path.exists(os.path.join(model_dir, 'deepfake_model.h5'))}")
print(f"Keras saved: {os.path.exists(os.path.join(model_dir, 'deepfake_model.keras'))}")

HDF5 saved: True
Keras saved: True
